In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Static Portfolio Optimization Strategies:

## Data Cleaning:

In [2]:
optTrain = pd.read_csv('df_train_no_nan.csv')
optTrain.index = optTrain['Date-Time']
optTrain = optTrain.drop('Date-Time', axis='columns')

lessLiquid = pd.read_csv('df_train_with_nan.csv')
lessLiquid.index = lessLiquid['Date-Time']
lessLiquid = lessLiquid.drop('Date-Time', axis='columns')

optTest = pd.read_csv('df_test.csv')
optTest.index = optTest['Date-Time']
optTest = optTest.drop('Date-Time', axis='columns')
optTest.head()

,SPYE221011300.U,SPYE221011400.U,SPYE221011500.U,SPYE221011600.U,SPYE221011700.U,SPYE221011800.U,SPYE221011900.U,SPYE221012000.U,SPYE221012100.U,SPYF191011600.U,...,SPYQ221011000.U,SPYQ221011200.U,SPYQ221011300.U,SPYQ221011400.U,SPYQ221011500.U,SPYQ221011600.U,SPYQ221011700.U,SPYQ221011800.U,SPYR191011500.U,SPYR191011800.U
Date-Time,,,,,,,,,,,,,,,,,,,,,
2010-05-06T09:30:00.000000000-04,0.015470,0.020000,0.028146,0.027311,0.033520,0.022989,0.034286,0.026316,0.014286,0.019444,...,-0.047619,-0.033981,-0.024691,-0.030822,-0.028653,-0.028640,-0.027833,-0.023372,-0.015408,-0.018620
2010-05-06T09:35:00.000000000-04,-0.015005,-0.020487,0.004847,0.004090,0.000000,0.015094,0.027778,-0.032520,0.000000,0.000000,...,0.000000,0.000000,0.034632,-0.003497,0.000000,0.004902,0.010267,0.000000,0.007937,-0.004459
2010-05-06T09:40:00.000000000-04,0.057734,0.057366,0.055380,0.064386,0.075472,0.092937,0.097297,0.100840,0.112676,0.046448,...,-0.114286,-0.125000,-0.133891,-0.106007,-0.117302,-0.104878,-0.101833,-0.097603,-0.065183,-0.055493
2010-05-06T09:45:00.000000000-04,-0.056295,-0.058752,-0.079764,-0.082552,-0.079012,-0.101010,-0.117073,-0.135338,-0.126582,-0.049479,...,0.048387,0.106509,0.097561,0.089069,0.079734,0.083102,0.072893,0.086042,0.063793,0.058252
2010-05-06T09:50:00.000000000-04,-0.009657,-0.020645,-0.025357,-0.026157,-0.029255,-0.036900,-0.032432,-0.025210,-0.028169,-0.018945,...,0.046512,0.038251,0.045249,0.029963,0.024768,0.030691,0.025586,0.039356,0.024590,0.017361


In [3]:
def fillna_avg(column):
    is_na = column.isna()
    if is_na[0]:
        column[0] = 0
    
    if is_na[is_na.shape[0]-1]:
        column[is_na.shape[0]-1] = 0
    
    for i in range(1, len(is_na)-1):
        if is_na[i]:
            column[i] = (column[i-1]+column[i+1])/2
    
    return column

In [4]:
lessLiquid = lessLiquid.apply(fillna_avg, axis=0)
optTest = optTest.iloc[:-3, :].apply(fillna_avg, axis=0)

Here, we can filter the data to create an investment universe of the top $n$ and bottom $n$ stocks by the options' implied volatility or their greeks such as delta, gamma, or theta. To keep the models simple, we are using the entire dataset for now.

## 1. Standard Markowitz Model

In [5]:
def stdPortfolio(mu: np.array, cov: np.array, target: float):
    one = np.ones(len(mu))
    A = one.T @ np.linalg.inv(cov) @ one
    B = one.T @ np.linalg.inv(cov) @ mu
    C = mu.T @ np.linalg.inv(cov) @ mu
    delta = A*C - B*B

    lam = (C - target*B)/delta
    gamma = (target*A - B)/delta
    w_opt = lam*np.linalg.inv(cov) @ one + gamma*np.linalg.inv(cov) @ mu
    return w_opt

In [6]:
# Training Data - most liquid options
target = 0.1
mu = np.mean(optTrain, axis=0)
cov = np.array(optTrain.cov())
w_opt = stdPortfolio(mu, cov, target)
trainRet = np.array(optTrain) @ w_opt


# Training Data - less liquid options
mu = np.mean(lessLiquid, axis=0)
cov = np.array(lessLiquid.cov())
w_opt = stdPortfolio(mu, cov, target)
ll_ret = np.array(lessLiquid) @ w_opt


# Testing Data - May 6th, 2010
mu = np.mean(optTest, axis=0)
cov = np.array(optTest.cov())
w_opt = stdPortfolio(mu, cov, target)
testRet = np.array(optTest) @ w_opt

In [7]:
cumul_ret = [np.cumsum(trainRet)[-1], np.cumsum(ll_ret)[-1], np.cumsum(testRet)[-1]]
avg_returns = np.array([np.mean(trainRet), np.mean(ll_ret), np.mean(testRet)])
vols = np.array([np.std(trainRet), np.std(ll_ret), np.std(testRet)])
sharpe = avg_returns/vols
performanceSummary = pd.DataFrame({'Total Return': cumul_ret, 'Avg Return': avg_returns, 'Volatility': vols, 
                      'Sharpe Ratio': sharpe}, index=['Most Liquid', 'Less Liquid', 'Testing'])
performanceSummary

,Total Return,Avg Return,Volatility,Sharpe Ratio
Most Liquid,18.4,0.1,0.363117,0.275393
Less Liquid,18.4,0.1,0.207741,0.481369
Testing,8.2,0.1,0.197581,0.506123


## 2. Portfolio with Risk-free Rate

In [8]:
def rfrPortfolio(mu: np.array, cov: np.array, target: float, rfr: float):
    one = np.ones(len(mu))
    A = one.T @ np.linalg.inv(cov) @ one
    B = one.T @ np.linalg.inv(cov) @ mu
    C = mu.T @ np.linalg.inv(cov) @ mu
    delta = A*C - B*B
    lam = (C - target*B)/delta
    gamma = (target*A - B)/delta
    C = (target - rfr)/(np.transpose(mu-rfr*one) @ np.linalg.inv(cov) @ mu-rfr*one)
    w_opt = np.matmul(C*np.linalg.inv(cov), mu-rfr*one)
    return w_opt

In [9]:
# Most Liquid
rfr = 0.03
mu = np.mean(optTrain, axis=0)
cov = np.array(optTrain.cov())
w_opt = rfrPortfolio(mu, cov, target, rfr)
trainRet = np.array(optTrain) @ w_opt


# Less Liquid
mu = np.mean(lessLiquid, axis=0)
cov = np.array(lessLiquid.cov())
w_opt = rfrPortfolio(mu, cov, target, rfr)
ll_ret = np.array(lessLiquid) @ w_opt


# Testing Data
mu = np.mean(optTest, axis=0)
cov = np.array(optTest.cov())
w_opt = rfrPortfolio(mu, cov, target, rfr)
testRet = np.array(optTest) @ w_opt

In [10]:
cumul_ret = [np.cumsum(trainRet)[-1], np.cumsum(ll_ret)[-1], np.cumsum(testRet)[-1]]
avg_returns = np.array([np.mean(trainRet), np.mean(ll_ret), np.mean(testRet)])
vols = np.array([np.std(trainRet), np.std(ll_ret), np.std(testRet)])
sharpe = avg_returns/vols
performanceSummary = pd.DataFrame({'Total Return': cumul_ret, 'Avg Return': avg_returns, 'Volatility': vols, 
                      'Sharpe Ratio': sharpe}, index=['Most Liquid', 'Less Liquid', 'Testing'])
performanceSummary

,Total Return,Avg Return,Volatility,Sharpe Ratio
Most Liquid,-18.533517,-0.100726,22.678664,-0.004441
Less Liquid,13.617763,0.074010,0.745664,0.099253
Testing,5.913533,0.072116,0.287956,0.250442


## 3. Portfolio with Shrinkage

In [11]:
def portShrinkage(mu: np.array, cov: np.array, target: float):
    # Mean vector:
    N = len(mu)
    T = 10000
    target = 0.1
    one = np.ones(N)
    mu_g = (one @ np.linalg.inv(cov) @ mu)/(one @ np.linalg.inv(cov) @ one)
    omega = (N+2)/(N + 2 + T*(mu - mu_g*one).T @ np.linalg.inv(cov) @ (mu - mu_g*one))
    mu_JS = (1 - omega)*mu + omega*target*one

    # Covariance matrix:
    LAMBDA = np.sqrt(np.diag(np.diag(cov)))
    C = np.linalg.inv(LAMBDA) @ cov @ np.linalg.inv(LAMBDA.T)
    rho = (2/(N*(N-1)))*np.triu(C).sum()
    C_CC = np.where(C != 1, rho, C)
    cov_CC = LAMBDA @ C_CC @ LAMBDA.T
    cov_LW = omega*cov_CC + (1-omega)*cov
    
    w_opt = stdPortfolio(mu_JS, cov_LW, target)
    return w_opt

In [12]:
# Most Liquid
mu = np.mean(optTrain, axis=0)
cov = np.array(optTrain.cov())
w_opt = portShrinkage(mu, cov, target)
trainRet = np.array(optTrain) @ w_opt


# Less Liquid
mu = np.mean(lessLiquid, axis=0)
cov = np.array(lessLiquid.cov())
w_opt = portShrinkage(mu, cov, target)
ll_ret = np.array(lessLiquid) @ w_opt


# Testing Data
mu = np.mean(optTest, axis=0)
cov = np.array(optTest.cov())
w_opt = portShrinkage(mu, cov, target)
testRet = np.array(optTest) @ w_opt

In [13]:
cumul_ret = [np.cumsum(trainRet)[-1], np.cumsum(ll_ret)[-1], np.cumsum(testRet)[-1]]
avg_returns = np.array([np.mean(trainRet), np.mean(ll_ret), np.mean(testRet)])
vols = np.array([np.std(trainRet), np.std(ll_ret), np.std(testRet)])
sharpe = avg_returns/vols
performanceSummary = pd.DataFrame({'Total Return': cumul_ret, 'Avg Return': avg_returns, 'Volatility': vols, 
                      'Sharpe Ratio': sharpe}, index=['Most Liquid', 'Less Liquid', 'Testing'])
performanceSummary

,Total Return,Avg Return,Volatility,Sharpe Ratio
Most Liquid,18.4,0.1,0.403658,0.247734
Less Liquid,18.4,0.1,0.224499,0.445436
Testing,8.2,0.1,0.208075,0.480596


## 4. Robust Optimization

Utilizing an uncertainty set with $\delta = 1.5$ and maximum portfolio variance of 0.0005.

In [14]:
import rsome as rso
from rsome import ro
from rsome import grb_solver as grb

In [27]:
def robustPortfolio(mu: np.array, cov: np.array, target: float):
    n = len(mu)
    one = np.ones(n)
    LAMBDA = 2
    delta = 1.5*one
    A = one.T @ np.linalg.inv(cov) @ one
    B = one.T @ np.linalg.inv(cov) @ mu
    gamma = (target*A - B)/delta

    model = ro.Model()
    x = model.dvar(n)
    z = model.rvar(n)
    Q = np.diag(np.diag(cov))
    model.maxmin(np.array(mu) @ x - delta @ x - LAMBDA*rso.quad(x, Q),
                 rso.norm(z, np.infty) <= 1,
                 rso.norm(z, 1) <= gamma)
    model.st(rso.quad(x, Q) <= 5e-4)
    model.st(x.sum() == 1)
    model.st(x >= 0)
    model.solve(grb)
    w_opt = x.get()
    return w_opt

In [28]:
# Most Liquid
mu = np.mean(optTrain, axis=0)
cov = np.array(optTrain.cov())
w_opt = robustPortfolio(mu, cov, target)
trainRet = np.array(optTrain) @ w_opt


# Less Liquid
mu = np.mean(lessLiquid, axis=0)
cov = np.array(lessLiquid.cov())
w_opt = robustPortfolio(mu, cov, target)
ll_ret = np.array(lessLiquid) @ w_opt


# Testing Data
mu = np.mean(optTest, axis=0)
cov = np.array(optTest.cov())
w_opt = robustPortfolio(mu, cov, target)
testRet = np.array(optTest) @ w_opt

Being solved by Gurobi...
Solution status: 2
Running time: 0.0080s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0080s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0000s


In [29]:
cumul_ret = [np.cumsum(trainRet)[-1], np.cumsum(ll_ret)[-1], np.cumsum(testRet)[-1]]
avg_returns = np.array([np.mean(trainRet), np.mean(ll_ret), np.mean(testRet)])
vols = np.array([np.std(trainRet), np.std(ll_ret), np.std(testRet)])
sharpe = avg_returns/vols
performanceSummary = pd.DataFrame({'Total Return': cumul_ret, 'Avg Return': avg_returns, 'Volatility': vols, 
                      'Sharpe Ratio': sharpe}, index=['Most Liquid', 'Less Liquid', 'Testing'])
performanceSummary

,Total Return,Avg Return,Volatility,Sharpe Ratio
Most Liquid,0.267620,0.001454,0.061304,0.023725
Less Liquid,0.341498,0.001856,0.059689,0.031094
Testing,0.789215,0.009625,0.053024,0.181514
